In [2]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Parameters
img_width, img_height = 128, 128
batch_size = 32
epochs = 50

# Directories
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Data generator for training data

# Data generator for training data with validation split
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # 20% validation split

# Separate generator for training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')  # Specify 'training' subset

# Separate generator for validation data
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  # Specify 'validation' subset



# Data generator for test data
test_files = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]

# Create a DataFrame with the filenames
test_df = pd.DataFrame({"Filename": test_files})

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='Filename',
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)


Found 7920 images belonging to 5 classes.
Found 1980 images belonging to 5 classes.
Found 100 validated image filenames.


In [46]:
tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [2]:
###################    INCEPTION

from tensorflow.keras import layers, Model, Input

# Define a simple Inception-like module
def inception_module(x, filters):
    conv1x1 = layers.Conv2D(filters=filters[0], kernel_size=(1, 1), padding='same', activation='relu')(x)

    conv3x3_reduce = layers.Conv2D(filters=filters[1], kernel_size=(1, 1), padding='same', activation='relu')(x)
    conv3x3 = layers.Conv2D(filters=filters[2], kernel_size=(3, 3), padding='same', activation='relu')(conv3x3_reduce)

    conv5x5_reduce = layers.Conv2D(filters=filters[3], kernel_size=(1, 1), padding='same', activation='relu')(x)
    conv5x5 = layers.Conv2D(filters=filters[4], kernel_size=(5, 5), padding='same', activation='relu')(conv5x5_reduce)

    maxpool = layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    maxpool_proj = layers.Conv2D(filters=filters[5], kernel_size=(1, 1), padding='same', activation='relu')(maxpool)

    output = layers.concatenate([conv1x1, conv3x3, conv5x5, maxpool_proj], axis=-1)
    return output

# Create a small Inception-like network using the above module
input_layer = Input(shape=(128, 128, 3))

# Simple stack of inception modules
x = inception_module(input_layer, filters=[16, 16, 16, 16, 16, 16])
x = inception_module(x, filters=[32, 32, 32, 32, 32, 32])
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = inception_module(x, filters=[64, 64, 64, 64, 64, 64])
x = layers.GlobalAveragePooling2D()(x)
output_layer = layers.Dense(5, activation='softmax')(x)  # Adjust num_classes as per your dataset

# Create the model
inception_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
inception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
inception_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 16  64          ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_3 (Conv2D)              (None, 128, 128, 16  64          ['input_1[0][0]']                
                                )                                                             

In [3]:
inception_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator)

Epoch 1/50
248/248 [==============================] - 47s 157ms/step - loss: 1.3099 - accuracy: 0.4295 - val_loss: 1.3182 - val_accuracy: 0.4354
Epoch 2/50
248/248 [==============================] - 36s 144ms/step - loss: 1.1469 - accuracy: 0.5534 - val_loss: 1.3118 - val_accuracy: 0.4682
Epoch 3/50
248/248 [==============================] - 36s 144ms/step - loss: 1.1079 - accuracy: 0.5722 - val_loss: 1.2982 - val_accuracy: 0.4318
Epoch 4/50
248/248 [==============================] - 37s 148ms/step - loss: 1.0823 - accuracy: 0.5872 - val_loss: 1.2527 - val_accuracy: 0.4985
Epoch 5/50
248/248 [==============================] - 36s 144ms/step - loss: 1.0414 - accuracy: 0.6044 - val_loss: 1.2190 - val_accuracy: 0.5020
Epoch 6/50
248/248 [==============================] - 36s 144ms/step - loss: 1.0088 - accuracy: 0.6141 - val_loss: 1.1572 - val_accuracy: 0.5460
Epoch 7/50
248/248 [==============================] - 36s 145ms/step - loss: 0.9947 - accuracy: 0.6186 - val_loss: 1.1492 - val_ac

Epoch 1/50
248/248 [==============================] - 47s 157ms/step - loss: 1.3099 - accuracy: 0.4295 - val_loss: 1.3182 - val_accuracy: 0.4354
Epoch 2/50
248/248 [==============================] - 36s 144ms/step - loss: 1.1469 - accuracy: 0.5534 - val_loss: 1.3118 - val_accuracy: 0.4682
Epoch 3/50
248/248 [==============================] - 36s 144ms/step - loss: 1.1079 - accuracy: 0.5722 - val_loss: 1.2982 - val_accuracy: 0.4318
Epoch 4/50
248/248 [==============================] - 37s 148ms/step - loss: 1.0823 - accuracy: 0.5872 - val_loss: 1.2527 - val_accuracy: 0.4985
Epoch 5/50
248/248 [==============================] - 36s 144ms/step - loss: 1.0414 - accuracy: 0.6044 - val_loss: 1.2190 - val_accuracy: 0.5020
Epoch 6/50
248/248 [==============================] - 36s 144ms/step - loss: 1.0088 - accuracy: 0.6141 - val_loss: 1.1572 - val_accuracy: 0.5460
Epoch 7/50
248/248 [==============================] - 36s 145ms/step - loss: 0.9947 - accuracy: 0.6186 - val_loss: 1.1492 - val_accuracy: 0.5495
Epoch 8/50
248/248 [==============================] - 36s 144ms/step - loss: 0.9669 - accuracy: 0.6304 - val_loss: 1.1707 - val_accuracy: 0.5217
Epoch 9/50
248/248 [==============================] - 36s 144ms/step - loss: 0.9407 - accuracy: 0.6427 - val_loss: 1.1748 - val_accuracy: 0.5409
Epoch 10/50
248/248 [==============================] - 36s 144ms/step - loss: 0.9273 - accuracy: 0.6501 - val_loss: 1.1648 - val_accuracy: 0.5611
Epoch 11/50
248/248 [==============================] - 36s 144ms/step - loss: 0.9273 - accuracy: 0.6528 - val_loss: 1.1097 - val_accuracy: 0.5747
Epoch 12/50
248/248 [==============================] - 36s 145ms/step - loss: 0.9012 - accuracy: 0.6620 - val_loss: 1.0811 - val_accuracy: 0.5854
Epoch 13/50
248/248 [==============================] - 36s 144ms/step - loss: 0.8878 - accuracy: 0.6644 - val_loss: 1.1174 - val_accuracy: 0.5525
Epoch 14/50
248/248 [==============================] - 36s 144ms/step - loss: 0.8686 - accuracy: 0.6705 - val_loss: 1.0927 - val_accuracy: 0.5742
Epoch 15/50
248/248 [==============================] - 35s 143ms/step - loss: 0.8611 - accuracy: 0.6807 - val_loss: 1.1162 - val_accuracy: 0.5530
Epoch 16/50
248/248 [==============================] - 35s 143ms/step - loss: 0.8617 - accuracy: 0.6803 - val_loss: 1.0479 - val_accuracy: 0.5848
Epoch 17/50
248/248 [==============================] - 36s 143ms/step - loss: 0.8427 - accuracy: 0.6836 - val_loss: 1.0615 - val_accuracy: 0.5949
Epoch 18/50
248/248 [==============================] - 36s 143ms/step - loss: 0.8336 - accuracy: 0.6881 - val_loss: 0.9947 - val_accuracy: 0.6202
Epoch 19/50
248/248 [==============================] - 36s 143ms/step - loss: 0.8140 - accuracy: 0.6949 - val_loss: 1.0465 - val_accuracy: 0.6045
Epoch 20/50
248/248 [==============================] - 36s 143ms/step - loss: 0.8132 - accuracy: 0.7018 - val_loss: 1.0163 - val_accuracy: 0.6152
Epoch 21/50
248/248 [==============================] - 36s 144ms/step - loss: 0.8011 - accuracy: 0.7024 - val_loss: 0.9913 - val_accuracy: 0.6328
Epoch 22/50
248/248 [==============================] - 36s 143ms/step - loss: 0.7933 - accuracy: 0.7105 - val_loss: 1.0353 - val_accuracy: 0.6187
Epoch 23/50
248/248 [==============================] - 36s 143ms/step - loss: 0.7822 - accuracy: 0.7163 - val_loss: 1.0087 - val_accuracy: 0.6217
Epoch 24/50
248/248 [==============================] - 36s 144ms/step - loss: 0.7654 - accuracy: 0.7264 - val_loss: 1.0056 - val_accuracy: 0.6101
Epoch 25/50
248/248 [==============================] - 36s 143ms/step - loss: 0.7648 - accuracy: 0.7193 - val_loss: 0.9585 - val_accuracy: 0.6384
Epoch 26/50
248/248 [==============================] - 36s 145ms/step - loss: 0.7422 - accuracy: 0.7319 - val_loss: 1.0235 - val_accuracy: 0.6207
Epoch 27/50
248/248 [==============================] - 37s 147ms/step - loss: 0.7418 - accuracy: 0.7303 - val_loss: 1.0270 - val_accuracy: 0.6152
Epoch 28/50
248/248 [==============================] - 36s 143ms/step - loss: 0.7441 - accuracy: 0.7266 - val_loss: 0.9935 - val_accuracy: 0.6227
Epoch 29/50
248/248 [==============================] - 36s 145ms/step - loss: 0.7266 - accuracy: 0.7317 - val_loss: 0.9643 - val_accuracy: 0.6369
Epoch 30/50
248/248 [==============================] - 36s 146ms/step - loss: 0.7244 - accuracy: 0.7316 - val_loss: 1.0191 - val_accuracy: 0.6111
Epoch 31/50
248/248 [==============================] - 36s 144ms/step - loss: 0.7069 - accuracy: 0.7366 - val_loss: 1.0433 - val_accuracy: 0.6283
Epoch 32/50
248/248 [==============================] - 36s 144ms/step - loss: 0.7050 - accuracy: 0.7400 - val_loss: 0.9521 - val_accuracy: 0.6485
Epoch 33/50
248/248 [==============================] - 36s 143ms/step - loss: 0.6973 - accuracy: 0.7506 - val_loss: 0.9518 - val_accuracy: 0.6424
Epoch 34/50
248/248 [==============================] - 36s 144ms/step - loss: 0.6906 - accuracy: 0.7491 - val_loss: 0.9364 - val_accuracy: 0.6490
Epoch 35/50
248/248 [==============================] - 36s 145ms/step - loss: 0.6752 - accuracy: 0.7540 - val_loss: 0.9241 - val_accuracy: 0.6596
Epoch 36/50
248/248 [==============================] - 36s 143ms/step - loss: 0.6722 - accuracy: 0.7578 - val_loss: 0.9788 - val_accuracy: 0.6606
Epoch 37/50
248/248 [==============================] - 36s 145ms/step - loss: 0.6636 - accuracy: 0.7583 - val_loss: 0.9455 - val_accuracy: 0.6455
Epoch 38/50
248/248 [==============================] - 36s 145ms/step - loss: 0.6680 - accuracy: 0.7539 - val_loss: 0.9707 - val_accuracy: 0.6268
Epoch 39/50
248/248 [==============================] - 36s 145ms/step - loss: 0.6537 - accuracy: 0.7634 - val_loss: 0.9423 - val_accuracy: 0.6591
Epoch 40/50
248/248 [==============================] - 36s 144ms/step - loss: 0.6549 - accuracy: 0.7600 - val_loss: 0.9432 - val_accuracy: 0.6414
Epoch 41/50
248/248 [==============================] - 36s 144ms/step - loss: 0.6408 - accuracy: 0.7650 - val_loss: 0.9193 - val_accuracy: 0.6551
Epoch 42/50
248/248 [==============================] - 36s 144ms/step - loss: 0.6468 - accuracy: 0.7650 - val_loss: 0.9760 - val_accuracy: 0.6439
Epoch 43/50
248/248 [==============================] - 36s 144ms/step - loss: 0.6237 - accuracy: 0.7732 - val_loss: 0.9862 - val_accuracy: 0.6561
Epoch 44/50
248/248 [==============================] - 36s 144ms/step - loss: 0.6260 - accuracy: 0.7721 - val_loss: 0.9237 - val_accuracy: 0.6571
Epoch 45/50
248/248 [==============================] - 36s 145ms/step - loss: 0.6222 - accuracy: 0.7804 - val_loss: 0.9596 - val_accuracy: 0.6551
Epoch 46/50
248/248 [==============================] - 36s 145ms/step - loss: 0.6074 - accuracy: 0.7741 - val_loss: 0.9333 - val_accuracy: 0.6646
Epoch 47/50
248/248 [==============================] - 36s 144ms/step - loss: 0.6060 - accuracy: 0.7802 - val_loss: 0.9297 - val_accuracy: 0.6652
Epoch 48/50
248/248 [==============================] - 37s 148ms/step - loss: 0.6079 - accuracy: 0.7793 - val_loss: 0.9327 - val_accuracy: 0.6601
Epoch 49/50
248/248 [==============================] - 36s 144ms/step - loss: 0.5866 - accuracy: 0.7888 - val_loss: 0.9004 - val_accuracy: 0.6808
Epoch 50/50
248/248 [==============================] - 36s 144ms/step - loss: 0.5818 - accuracy: 0.7891 - val_loss: 0.9237 - val_accuracy: 0.6843


In [2]:
# DIFFERENT TRIALS
from tensorflow.keras import layers, Model, Input

# Define a modified Inception-like module
def inception_module(x, filters):
    conv1x1 = layers.Conv2D(filters=filters[0], kernel_size=(1, 1), padding='same', activation='relu')(x)

    conv3x3_reduce = layers.Conv2D(filters=filters[1], kernel_size=(1, 1), padding='same', activation='relu')(x)
    conv3x3 = layers.Conv2D(filters=filters[2], kernel_size=(3, 3), padding='same', activation='relu')(conv3x3_reduce)

    conv5x5_reduce = layers.Conv2D(filters=filters[3], kernel_size=(1, 1), padding='same', activation='relu')(x)
    conv5x5 = layers.Conv2D(filters=filters[4], kernel_size=(5, 5), padding='same', activation='relu')(conv5x5_reduce)

    maxpool = layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    maxpool_proj = layers.Conv2D(filters=filters[5], kernel_size=(1, 1), padding='same', activation='relu')(maxpool)

    output = layers.concatenate([conv1x1, conv3x3, conv5x5, maxpool_proj], axis=-1)
    return output

# Create a customized Inception-like network using the above module
input_layer = Input(shape=(128, 128, 3))

# Stacked inception modules
x = inception_module(input_layer, filters=[32, 32, 32, 32, 32, 32])
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = inception_module(x, filters=[64, 64, 64, 64, 64, 64])
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = inception_module(x, filters=[128, 128, 128, 128, 128, 128])
x = layers.GlobalAveragePooling2D()(x)
output_layer = layers.Dense(5, activation='softmax')(x)  # Adjust num_classes as per your dataset

# Create the model
modified_inception_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
modified_inception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
modified_inception_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 32  128         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_3 (Conv2D)              (None, 128, 128, 32  128         ['input_1[0][0]']                
                                )                                                             

In [5]:
modified_inception_model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator)

Epoch 1/50
248/248 [==============================] - 34s 135ms/step - loss: 0.9528 - accuracy: 0.6333 - val_loss: 1.1267 - val_accuracy: 0.5384
Epoch 2/50
248/248 [==============================] - 34s 135ms/step - loss: 0.9405 - accuracy: 0.6418 - val_loss: 1.0962 - val_accuracy: 0.5838
Epoch 3/50
248/248 [==============================] - 34s 138ms/step - loss: 0.9172 - accuracy: 0.6490 - val_loss: 1.1188 - val_accuracy: 0.5465
Epoch 4/50
248/248 [==============================] - 33s 133ms/step - loss: 0.8929 - accuracy: 0.6607 - val_loss: 1.0641 - val_accuracy: 0.5808
Epoch 5/50
248/248 [==============================] - 34s 135ms/step - loss: 0.8519 - accuracy: 0.6797 - val_loss: 1.0466 - val_accuracy: 0.5818
Epoch 6/50
248/248 [==============================] - 40s 160ms/step - loss: 0.8382 - accuracy: 0.6838 - val_loss: 1.0164 - val_accuracy: 0.5955
Epoch 7/50
248/248 [==============================] - 63s 254ms/step - loss: 0.8118 - accuracy: 0.6972 - val_loss: 1.0648 - val_ac

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Load a pretrained model (e.g., ResNet50 as an example)
pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze layers
for layer in pretrained_model.layers:
    layer.trainable = False

# Add custom classification layers
x = Flatten()(pretrained_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)  # Set num_classes to your desired number

# Create the model
model = tf.keras.models.Model(inputs=pretrained_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with fine-tuning
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    # Add other necessary parameters like batch size, callbacks, etc.
)


c:\Users\badr0\.conda\envs\py310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10


: 